### A few initial libraries ###

In [1]:
import numpy as np
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble
from qiskit.tools.monitor import job_monitor

from qiskit import IBMQ
from trios_bench.cnx_logdepth import generate_cnx_log_depth
from trios_bench.incrementer_borrowedbit import generate_incrementer

/Users/revilooliver/Documents/quantum_computing/anaconda/anaconda3/envs/qiskit-terra/lib/python3.7/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [2]:
from qiskit_superstaq.superstaq_provider import SuperstaQProvider

In [3]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.18.3', 'qiskit-aer': '0.9.1', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.18.1', 'qiskit-aqua': '0.9.5', 'qiskit': '0.32.1', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

### Load device and the device configuration object ###

In [4]:
IBMQ.load_account()

provider = IBMQ.get_provider('ibm-q-ornl')

In [5]:
superstaq_provider = SuperstaQProvider(api_key="ya29.A0ARrdaM-BWPjpkjhKgbVSRwWROaVE3g6mKCew2all4yXuPNLIwc6l65K10DiAMa-w8he0mR-i0Tb_jE6-GqqWlscj9Zq58rC2oSSzRVaLdZJMoy2bWqIXVhK3Tehp5N4zd8uHhyp01G9KC90Xmu3BKALiGiLP-w")

In [6]:
incrementer_qc = generate_incrementer(6)

In [7]:
incrementer_qc.draw()

┌───┐                                             »
q_0: ──■─────────■────■──┤ X ├──■───────────────────────────────────────■──»
       │  ┌───┐  │    │  └─┬─┘┌─┴─┐     ┌───┐               ┌───┐     ┌─┴─┐»
q_1: ──┼──┤ X ├──┼────┼────■──┤ X ├──■──┤ X ├──■─────────■──┤ X ├─────┤ X ├»
       │  ├───┤  │    │    │  └─┬─┘  │  └─┬─┘┌─┴─┐     ┌─┴─┐└─┬─┘     └─┬─┘»
q_2: ──┼──┤ X ├──┼────┼────┼────┼────┼────■──┤ X ├──■──┤ X ├──■────■────┼──»
       │  └───┘┌─┴─┐  │    │    │  ┌─┴─┐  │  └─┬─┘  │  └─┬─┘  │  ┌─┴─┐  │  »
q_3: ──┼───────┤ X ├──┼────┼────┼──┤ X ├──■────■────┼────■────■──┤ X ├──┼──»
       │  ┌───┐└───┘  │    │    │  └───┘          ┌─┴─┐          └───┘  │  »
q_4: ──┼──┤ X ├───────┼────┼────┼─────────────────┤ X ├─────────────────┼──»
     ┌─┴─┐└───┘     ┌─┴─┐  │    │                 └───┘                 │  »
q_5: ┤ X ├──────────┤ X ├──■────■───────────────────────────────────────■──»
     └───┘          └───┘                                                  »
«     ┌───┐                    ┌───┐                                        »
«q_0: ┤ X ├─────────────────■──┤ X ├──■─────────────────────────────────────»
«     └─┬─┘          ┌───┐  │  └─┬─┘┌─┴─┐     ┌───┐               ┌───┐     »
«q_1: ──■─────────■──┤ X ├──┼────■──┤ X ├──■──┤ X ├──■─────────■──┤ X ├─────»
«       │  ┌───┐  │  └───┘  │    │  └─┬─┘  │  └─┬─┘┌─┴─┐     ┌─┴─┐└─┬─┘     »
«q_2: ──┼──┤ X ├──┼─────────┼────┼────┼────┼────■──┤ X ├──■──┤ X ├──■────■──»
«       │  └───┘  │         │    │    │  ┌─┴─┐  │  └─┬─┘  │  └─┬─┘  │  ┌─┴─┐»
«q_3: ──┼─────────┼─────────┼────┼────┼──┤ X ├──■────■────┼────■────■──┤ X ├»
«       │         │         │    │    │  └───┘          ┌─┴─┐          └───┘»
«q_4: ──┼─────────┼─────────┼────┼────┼─────────────────┤ X ├───────────────»
«       │       ┌─┴─┐     ┌─┴─┐  │    │                 └───┘               »
«q_5: ──■───────┤ X ├─────┤ X ├──■────■─────────────────────────────────────»
«               └───┘     └───┘                                             »
«          ┌───┐                                                            »
«q_0: ──■──┤ X ├───────■────■───────────────────■─────────■─────────────────»
«     ┌─┴─┐└─┬─┘       │    │                   │         │  ┌───┐          »
«q_1: ┤ X ├──■────■────┼────┼───────────────────■─────────■──┤ X ├──────────»
«     └─┬─┘  │    │    │    │                   │         │  ├───┤          »
«q_2: ──┼────┼────┼────┼────┼──────────────■────┼────■────┼──┤ X ├──────────»
«       │    │    │    │  ┌─┴─┐┌───┐       │  ┌─┴─┐  │  ┌─┴─┐├───┤┌───┐     »
«q_3: ──┼────┼────┼────┼──┤ X ├┤ X ├───────■──┤ X ├──■──┤ X ├┤ X ├┤ X ├─────»
«       │    │    │    │  └───┘└─┬─┘┌───┐  │  └───┘  │  └───┘└─┬─┘├───┤┌───┐»
«q_4: ──┼────┼────┼────┼─────────┼──┤ X ├──┼─────────┼─────────┼──┤ X ├┤ X ├»
«       │    │  ┌─┴─┐┌─┴─┐┌───┐  │  └─┬─┘┌─┴─┐     ┌─┴─┐       │  └─┬─┘└───┘»
«q_5: ──■────■──┤ X ├┤ X ├┤ X ├──■────■──┤ X ├─────┤ X ├───────■────■───────»
«               └───┘└───┘└───┘          └───┘     └───┘                    »
«                    ┌───┐                                             ┌───┐»
«q_0: ──■────■────■──┤ X ├──■───────────────────────────────────────■──┤ X ├»
«       │    │    │  └─┬─┘┌─┴─┐     ┌───┐               ┌───┐     ┌─┴─┐└─┬─┘»
«q_1: ──┼────┼────┼────■──┤ X ├──■──┤ X ├──■─────────■──┤ X ├─────┤ X ├──■──»
«       │    │    │    │  └─┬─┘  │  └─┬─┘┌─┴─┐     ┌─┴─┐└─┬─┘     └─┬─┘  │  »
«q_2: ──┼────┼────┼────┼────┼────┼────■──┤ X ├──■──┤ X ├──■────■────┼────┼──»
«       │  ┌─┴─┐  │    │    │  ┌─┴─┐  │  └─┬─┘  │  └─┬─┘  │  ┌─┴─┐  │    │  »
«q_3: ──┼──┤ X ├──┼────┼────┼──┤ X ├──■────■────┼────■────■──┤ X ├──┼────┼──»
«       │  ├───┤  │    │    │  └───┘          ┌─┴─┐          └───┘  │    │  »
«q_4: ──┼──┤ X ├──┼────┼────┼─────────────────┤ X ├─────────────────┼────┼──»
«     ┌─┴─┐└───┘┌─┴─┐  │    │                 └───┘                 │    │  »
«q_5: ┤ X ├─────┤ X ├──■────■───────────────────────────────────────■────■──»
«     └───┘     └───┘                                       

In [9]:
backend = provider.get_backend('ibmq_jakarta')

In [18]:
trans_qc = transpile(incrementer_qc, backend, optimization_level = 3)

In [19]:
trans_qc.count_ops()

OrderedDict([('rz', 889), ('cx', 800), ('sx', 469), ('x', 95)])

In [20]:
trans_qc.draw()

┌───┐       ┌───┐                                        »
      q_5 -> 0 ──■──┤ X ├──■────┤ X ├────────────────────────────────────────»
               ┌─┴─┐└─┬─┘┌─┴─┐  └───┘   ┌───┐        ┌───┐           ┌───┐   »
      q_4 -> 1 ┤ X ├──■──┤ X ├────■─────┤ X ├───■────┤ X ├─────■─────┤ X ├───»
               └───┘     └───┘    │     └─┬─┘   │    └───┘   ┌─┴─┐   └─┬─┘   »
      q_3 -> 2 ───────────────────┼───────┼─────┼────────────┤ X ├─────■─────»
                                ┌─┴─┐     │   ┌─┴─┐┌────────┐└───┘   ┌────┐  »
      q_2 -> 3 ─────────────────┤ X ├─────■───┤ X ├┤ Rz(-π) ├──■─────┤ √X ├──»
                    ┌───┐       ├───┤         └───┘└────────┘  │     └────┘  »
      q_0 -> 4 ──■──┤ X ├──■────┤ X ├──────────────────────────┼─────────────»
               ┌─┴─┐└─┬─┘┌─┴─┐┌─┴───┴──┐┌────┐               ┌─┴─┐┌─────────┐»
      q_1 -> 5 ┤ X ├──■──┤ X ├┤ Rz(-π) ├┤ √X ├───────────────┤ X ├┤ Rz(π/2) ├»
               └───┘     └───┘└────────┘└────┘               └───┘└─────────┘»
ancilla_0 -> 6 ──────────────────────────────────────────────────────────────»
                                                                             »
«                                                                           »
«      q_5 -> 0 ────────────────────────────────────────────────────────────»
«                                                                      ┌───┐»
«      q_4 -> 1 ─────■─────────────────────────────────────────────────┤ X ├»
«                  ┌─┴─┐                                               └─┬─┘»
«      q_3 -> 2 ───┤ X ├─────────────────────────────────────────────────┼──»
«               ┌──┴───┴───┐┌────┐     ┌──────────┐┌────┐┌────────────┐  │  »
«      q_2 -> 3 ┤ Rz(-π/2) ├┤ √X ├──■──┤ Rz(-π/2) ├┤ √X ├┤ Rz(1.5643) ├──■──»
«               └──────────┘└────┘  │  └──────────┘└────┘└────────────┘     »
«      q_0 -> 4 ────────────────────┼───────────────────────────────────────»
«                                 ┌─┴─┐┌──────────┐┌────┐  ┌────────┐       »
«      q_1 -> 5 ──────────────────┤ X ├┤ Rz(-π/2) ├┤ √X ├──┤ Rz(-π) ├───────»
«                                 └───┘└──────────┘└────┘  └────────┘       »
«ancilla_0 -> 6 ────────────────────────────────────────────────────────────»
«                                                                           »
«                                                                            »
«      q_5 -> 0 ─────────────────────────────────────────────────────────────»
«               ┌────────┐   ┌────┐   ┌────────┐                             »
«      q_4 -> 1 ┤ Rz(-π) ├───┤ √X ├───┤ Rz(-π) ├─────────────────────────────»
«               └────────┘   └────┘   └────────┘                             »
«      q_3 -> 2 ─────────────────────────────────────────────────────────────»
«                 ┌────┐  ┌──────────┐             ┌────┐  ┌──────────┐┌────┐»
«      q_2 -> 3 ──┤ √X ├──┤ Rz(-π/2) ├────■────────┤ √X ├──┤ Rz(-π/2) ├┤ √X ├»
«                 └────┘  └──────────┘    │        └────┘  └──────────┘└────┘»
«      q_0 -> 4 ──────────────────────────┼──────────────────────────────────»
«                                       ┌─┴─┐   ┌─────────┐                  »
«      q_1 -> 5 ────────────────────────┤ X ├───┤ Rz(π/2) ├──────────────────»
«                                       └───┘   └─────────┘                  »
«ancilla_0 -> 6 ─────────────────────────────────────────────────────────────»
«                                                                            »
«                                                                         »
«      q_5 -> 0 ──────────────────────────────────────────────────────────»
«                                                                         »
«      q_4 -> 1 ──────────────────────────────────────────────────────────»
«                                                                         »
«      q_3 -> 2 ──────────────────────────────────────────────────────────»
«                    ┌──

In [21]:
incrementer_out = superstaq_provider.ibmq_compile(trans_qc, target="ibmq_jakarta_qpu")

SuperstaQException: Status code: 502, Message: 'Non-retriable error making request to SuperstaQ API, Bad Gateway'